# Bicycle Facilities Reconciliation
This notebook is for determining the approximate build date (when facility was available for use) of bike facilities.

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import matching_script
import json

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

## Import the 2023 OSM bicycle network


In [ ]:
cycleways_osm = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='osm_cycleways')
cycleways_osm = cycleways_osm.loc[cycleways_osm['year']=='2023',['osmid','highway','name','facility_fwd','facility_rev','geometry']]

# Suggested Matches Script
Buffers the OSM cycleways and intersects it with the other bicycle inventory file. Check the names and the similarity of the intersected features to the original features. Results should be manually verified in QGIS.

### Settings

In [ ]:
buffer_ft = 100 # buffer the osm cycleways by this much
max_hausdorff_dist_ft = 3000 # if above this, reject a match

In [ ]:
#import processed versions
coa = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='coa')
arc = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='arc')


In [ ]:
#perform overlap
from importlib import reload
reload(matching_script)
coa_overlap = matching_script.suggested_matches(cycleways_osm,coa,'coa',buffer_ft,max_hausdorff_dist_ft)
arc_overlap = matching_script.suggested_matches(cycleways_osm,arc,'arc',buffer_ft,max_hausdorff_dist_ft)

#export
coa_overlap.to_file(config['bicycle_facilities_fp']/'suggested_matches.gpkg',layer='coa')
arc_overlap.to_file(config['bicycle_facilities_fp']/'suggested_matches.gpkg',layer='arc')

In [ ]:
print('Total Matches:')
print('coa:',coa_overlap.shape[0],'arc:',arc_overlap.shape[0])
print('Undecided:')
print('coa:',coa_overlap['auto_match'].isna().sum(),'arc:',arc_overlap['auto_match'].isna().sum())
print('Accept:')
print('coa:',(coa_overlap['auto_match']==1).sum(),'arc:',(arc_overlap['auto_match']==1).sum())
print('Reject:')
print('coa:',(coa_overlap['auto_match']==0).sum(),'arc:',(arc_overlap['auto_match']==0).sum())

# Missing Infrastructure (ID version)
Find the coa/arc features that were not considered in the overlap step

In [ ]:
#load the suggested infra
coa_overlap = gpd.read_file(config['bicycle_facilities_fp']/'suggested_matches.gpkg',layer='coa')
arc_overlap = gpd.read_file(config['bicycle_facilities_fp']/'suggested_matches.gpkg',layer='arc')


In [ ]:
#get ids of accepted matches
suggested_coa_ids = coa_overlap.loc[coa_overlap['auto_match'] == 1,'coa_id'].unique().tolist()
suggested_arc_ids = arc_overlap.loc[arc_overlap['auto_match'] == 1,'arc_id'].unique().tolist()

In [ ]:
#import raw versions
coa = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='coa')
arc = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='arc')
#garber = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='garber')

In [ ]:
#what's not covered
coa_inv = coa[coa['coa_id'].isin(suggested_coa_ids) == False].copy()
arc_inv = arc[arc['arc_id'].isin(suggested_arc_ids) == False].copy()
print(coa_inv.shape[0],arc_inv.shape[0],'not covered by osm cycleways')

In [ ]:
coa_inv['include'] = None
coa_inv['suggested_osmid'] = None # used to indicate which osmid this should be assigned to
coa_inv['notes'] = None
coa_inv.to_file(config['bicycle_facilities_fp']/'missing.gpkg',layer='coa')

arc_inv['include'] = None
arc_inv['suggested_osmid'] = None
arc_inv['notes'] = None
arc_inv.to_file(config['bicycle_facilities_fp']/'missing.gpkg',layer='arc')

# Reference OSM network if suggested matches is incorrect

In [ ]:
raw_osm = gpd.read_file(config['project_fp']/"OSM_Download/osm_2023.gpkg",layer='raw')

raw_osm['suggested_arc_match'] = None
raw_osm['suggested_coa_match'] = None
raw_osm['notes'] = None

raw_osm.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer="osm_2023")
del raw_osm

In [ ]:
network_osm = gpd.read_file(config['project_fp']/"Network/networks.gpkg",layer='osm_links')

network_osm['suggested_arc_match'] = None
network_osm['suggested_coa_match'] = None
network_osm['notes'] = None

network_osm.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer="osm_network_2023")
del network_osm

# Split into new file after this

# Add dates to OSM cycleways network

In [ ]:
cycleways_osm = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='osm_cycleways')
cycleways_osm = cycleways_osm.loc[cycleways_osm['year']=='2023',['osmid','highway','name','facility_fwd','facility_rev','geometry']]

In [ ]:
coa_overlap = gpd.read_file(config['bicycle_facilities_fp']/'suggested_matches.gpkg',layer='coa')
arc_overlap = gpd.read_file(config['bicycle_facilities_fp']/'suggested_matches.gpkg',layer='arc')

In [ ]:
# merge_cols = ['coa_id', 'coa_facilitytype', 'coa_builtby', 'coa_name',
#        'coa_onroad', 'coa_bothside', 'coa_year', 'coa_need_date',
#        'coa_osm_type', 'osmid']
accepted_coa = coa_overlap.loc[coa_overlap['auto_match']=='1',['osmid','coa_year']]
cycleways_osm_coa = pd.merge(cycleways_osm,accepted_coa,on='osmid',how='left')

In [ ]:
# merge_cols = ['arc_id', 'arc_name', 'arc_spec', 'arc_width', 'arc_material',
#        'arc_year', 'arc_need_date', 'arc_osm_type', 'osmid']
accepted_arc = arc_overlap.loc[arc_overlap['auto_match']=='1',['osmid','arc_year']]
cycleways_osm_coa_arc = pd.merge(cycleways_osm_coa,accepted_arc,on='osmid',how='left')

In [ ]:
cycleways_osm_coa_arc[cycleways_osm_coa_arc[['arc_year','coa_year']].notnull().any(axis=1)].explore()

In [ ]:
import numpy as np

#if both coa and arc date, take the coa date, otherwise arc
def date_fix(row):
    if row['coa_year'] != row['arc_year']:
        return row['coa_year']
    else:
        return row['arc_year']
cycleways_osm_coa_arc['year'] = cycleways_osm_coa_arc.apply(lambda row: date_fix(row),axis=1)
cycleways_osm_coa_arc.loc[cycleways_osm_coa_arc['year'].isna(),'year'] = np.nan

In [ ]:
# find the osm earliest appearance, if it's post 2020 then it definitely wasn't in the study area

In [ ]:
#TODO Set certain facilities to year=0 if it's not a bike lane or PATH trail (i.e., facilities not present in the newer data sources)
cycleways_osm_coa_arc.columns

In [ ]:
# cycleways_osm_coa_arc['min_date'] = cycleways_osm_coa_arc[['arc_year','coa_year']].min(axis=1)
# cycleways_osm_coa_arc['max_date'] = cycleways_osm_coa_arc[['arc_year','coa_year']].max(axis=1)

# #set null value
# cycleways_osm_coa_arc.loc[cycleways_osm_coa_arc['min_date'].isna(),'min_date'] = 0
# cycleways_osm_coa_arc.loc[cycleways_osm_coa_arc['max_date'].isna(),'max_date'] = 0

In [ ]:
#network_osm = gpd.read_file(Path(config['project_directory'])/f"Network/networks.gpkg",layer='osm_links')

In [ ]:
cycleways_osm_coa_arc[['facility_fwd','facility_rev']].value_counts()

# Export final

In [ ]:
cycleways_osm_coa_arc.to_file(config['bicycle_facilities_fp']/'osm_cycleways_w_dates.gpkg',layer='test_dates')

# Add reference osm files

In [ ]:
# years = [2014,2015,2016,2023]

# for year in years:
#     raw_osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_{year}.gpkg",layer='raw')
#     raw_osm.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer=f"osm_{year}")
#     del raw_osm

# Create a version of OSM for editing and adding data to (only run once)

In [ ]:
# osm = gpd.read_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='osm_2023')
# overwrite = False
# if overwrite:
#     osm.to_file(config['bicycle_facilities_fp']/'osm_edit.gpkg')

In [ ]:
# <!-- # Missing Infrastructure (Geometry Version)
# buffer_ft = 100
# overwrite_diff = False
# confirm_diff = False
# # import the 2023 OSM bicycle network
# cycleways_osm = gpd.read_file(config['bicycle_facilities_fp']/'osm_cycleways.gpkg')
# cycleways_osm = cycleways_osm.loc[cycleways_osm['year']=='2023']

# # get unary union of all features after buffering
# cycleways_osm_all = cycleways_osm.buffer(buffer_ft).unary_union
# arc_diff = arc[arc.geometry.intersects(cycleways_osm_all) == False]
# coa_diff = coa[coa.geometry.intersects(cycleways_osm_all) == False]
# garber_diff = garber[garber.geometry.intersects(cycleways_osm_all) == False]
# overwrite = False
# if overwrite:
#     coa_diff['valid_difference'] = None
#     coa_diff['notes'] = None
#     coa_diff.to_file(config['bicycle_facilities_fp']/'differences.gpkg',layer='coa')
    
#     arc_diff['valid_difference'] = None
#     arc_diff['notes'] = None
#     arc_diff.to_file(config['bicycle_facilities_fp']/'differences.gpkg',layer='arc')

#     garber_diff['valid_difference'] = None
#     garber_diff['notes'] = None
#     garber_diff.to_file(config['bicycle_facilities_fp']/'differences.gpkg',layer='garber')

# raw_osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_{config['geofabrik_year']}.gpkg",layer='raw')
# raw_osm.to_crs(config['projected_crs_epsg'],inplace=True)
# final_confirm = False
# if (overwrite_check(overwrite_diff,confirm_diff) == True) & (final_confirm == True):
#     raw_osm['arc_feature_id'] = None
#     raw_osm['coa_feature_id'] = None
#     raw_osm['garber_feature_id'] = None
#     raw_osm.to_file(config['bicycle_facilities_fp']/'differences.gpkg',layer='osm_edit')
# # raw_osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_{config['geofabrik_year']}.gpkg",layer='raw')
# # raw_osm.to_crs(config['projected_crs_epsg'],inplace=True)
# # final_confirm = True
# # if (overwrite_check(overwrite_diff,confirm_diff) == True) & (final_confirm == True):
# #     raw_osm['arc_feature_id'] = None
# #     raw_osm['coa_feature_id'] = None
# #     raw_osm['garber_feature_id'] = None
# #     raw_osm.to_file(config['bicycle_facilities_fp']/'differences.gpkg',layer='osm_edit')
# # #based on the 2023-01-01 Geofabrik Georgia Extract
# # #osm = gpd.read_file(Path(config['project_directory'])/'Network/networks.gpkg',layer='osm_links')
# # osm = gpd.read_file(Path(config['project_directory'])/f"OSM_Download/osm_2023.gpkg",layer='raw',ignore_geometry=True)
# # #osm = pd.merge(osm,raw_osm,on='osmid',how='left')

# # #create new fields for install dates
# # osm['install_year'] = None
# # osm['install_month'] = None
# # osm['install_day'] = None

# # #create new fields for updated fwd and rev infra types
# # osm['facility_fwd'] = None
# # osm['facility_rev'] = None

# # #create field for notes
# # osm['notes'] = None

# # #create field for edit date
# # osm['last_edited'] = None -->